# LightGBM 최적화

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('./data/train.csv').drop(columns=['UID'])
test_df = pd.read_csv('./data/test.csv').drop(columns=['UID'])

In [5]:
df.info()   # non-null: 이상치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   주거 형태              10000 non-null  object 
 1   연간 소득              10000 non-null  float64
 2   현재 직장 근속 연수        10000 non-null  object 
 3   체납 세금 압류 횟수        10000 non-null  float64
 4   개설된 신용계좌 수         10000 non-null  int64  
 5   신용 거래 연수           10000 non-null  float64
 6   최대 신용한도            10000 non-null  float64
 7   신용 문제 발생 횟수        10000 non-null  int64  
 8   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 9   개인 파산 횟수           10000 non-null  int64  
 10  대출 목적              10000 non-null  object 
 11  대출 상환 기간           10000 non-null  object 
 12  현재 대출 잔액           10000 non-null  float64
 13  현재 미상환 신용액         10000 non-null  float64
 14  월 상환 부채액           10000 non-null  float64
 15  신용 점수              10000 non-null  int64  
 16  채무 불이행 여부          1000

In [8]:
Y = df['채무 불이행 여부']
X = df.drop('채무 불이행 여부', axis=1)

print(X)

                주거 형태      연간 소득 현재 직장 근속 연수  체납 세금 압류 횟수  개설된 신용계좌 수  \
0                  자가  1941337.5      10년 이상          0.0           9   
1                  월세  1979505.0      10년 이상          0.0           5   
2                  월세  1356381.0          4년          0.0          12   
3                  월세  1049017.5          6년          0.0          15   
4                  월세  4320217.5          2년          0.0          11   
...               ...        ...         ...          ...         ...   
9995  주택 담보 대출 (거주 중)  1339473.0      10년 이상          0.0           9   
9996  주택 담보 대출 (거주 중)  2297230.5          2년          0.0          11   
9997  주택 담보 대출 (거주 중)  1221523.5      10년 이상          0.0           9   
9998               자가  3343584.0      10년 이상          0.0          10   
9999  주택 담보 대출 (거주 중)  2175133.5          5년          0.0           5   

      신용 거래 연수   최대 신용한도  신용 문제 발생 횟수  마지막 연체 이후 경과 개월 수  개인 파산 횟수   대출 목적  \
0         13.4  400597.5            0        

In [21]:
print(test_df)
print(test_df.columns)

                주거 형태      연간 소득 현재 직장 근속 연수  체납 세금 압류 횟수  개설된 신용계좌 수  \
0                  월세  1560090.0      10년 이상          0.0          13   
1     주택 담보 대출 (거주 중)  2102616.0          2년          0.0           9   
2     주택 담보 대출 (거주 중)  2477989.5      10년 이상          0.0          11   
3     주택 담보 대출 (거주 중)  1571091.0          6년          0.0           7   
4     주택 담보 대출 (거주 중)  2290260.0      10년 이상          0.0          19   
...               ...        ...         ...          ...         ...   
2057               월세   698107.5          5년          0.0           5   
2058  주택 담보 대출 (거주 중)  2092413.0      10년 이상          0.0          15   
2059               자가  2052000.0      10년 이상          0.0          20   
2060  주택 담보 대출 (거주 중)  4249777.5          6년          0.0          18   
2061               월세  1886415.0          8년          0.0          14   

      신용 거래 연수    최대 신용한도  신용 문제 발생 횟수  마지막 연체 이후 경과 개월 수  개인 파산 횟수   대출 목적  \
0         12.0   495561.0            0      

In [12]:
# 원 핫 인코딩
categories = ['주거 형태', '현재 직장 근속 연수', '대출 목적', '대출 상환 기간']


X = pd.get_dummies(X, columns = categories, dtype=int) 
test_X = pd.get_dummies(test_df, columns = categories, dtype=int) 

X

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,...,대출 목적_소규모 사업 자금,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_단기 상환,대출 상환 기간_장기 상환
0,1941337.5,0.0,9,13.4,400597.5,0,24,1,390903.0,225457.5,...,0,0,0,0,0,0,0,0,1,0
1,1979505.0,0.0,5,15.1,360679.5,0,11,0,1002184.5,64749.0,...,0,0,0,0,0,0,0,0,1,0
2,1356381.0,0.0,12,18.8,491770.5,1,74,3,227775.0,487644.0,...,0,0,0,0,0,0,0,0,1,0
3,1049017.5,0.0,15,14.8,411546.0,1,22,1,251383.5,413211.0,...,0,0,0,0,0,0,0,0,1,0
4,4320217.5,0.0,11,26.1,895288.5,0,32,0,1163176.5,78991.5,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1339473.0,0.0,9,18.7,319027.5,0,68,0,126216.0,177028.5,...,0,0,0,0,0,0,0,0,1,0
9996,2297230.5,0.0,11,28.3,399799.5,0,7,0,371907.0,347449.5,...,0,0,0,0,0,1,0,0,0,1
9997,1221523.5,0.0,9,30.1,823305.0,0,14,0,869736.0,176905.5,...,0,0,0,0,0,0,0,0,0,1
9998,3343584.0,0.0,10,20.3,724314.0,0,25,0,443008.5,139294.5,...,0,0,0,0,0,0,0,0,1,0


In [13]:
# 로그
cl = ['현재 대출 잔액', '현재 미상환 신용액', '월 상환 부채액']

for i in cl:
    X[i] = X[i].apply(lambda x: np.log1p(x))
    test_X[i] = test_X[i].apply(lambda x: np.log1p(x))

X

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,...,대출 목적_소규모 사업 자금,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_단기 상환,대출 상환 기간_장기 상환
0,1941337.5,0.0,9,13.4,400597.5,0,24,1,12.876217,12.325891,...,0,0,0,0,0,0,0,0,1,0
1,1979505.0,0.0,5,15.1,360679.5,0,11,0,13.817694,11.078289,...,0,0,0,0,0,0,0,0,1,0
2,1356381.0,0.0,12,18.8,491770.5,1,74,3,12.336118,13.097343,...,0,0,0,0,0,0,0,0,1,0
3,1049017.5,0.0,15,14.8,411546.0,1,22,1,12.434739,12.931716,...,0,0,0,0,0,0,0,0,1,0
4,4320217.5,0.0,11,26.1,895288.5,0,32,0,13.966666,11.277108,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1339473.0,0.0,9,18.7,319027.5,0,68,0,11.745758,12.084072,...,0,0,0,0,0,0,0,0,1,0
9996,2297230.5,0.0,11,28.3,399799.5,0,7,0,12.826402,12.758377,...,0,0,0,0,0,1,0,0,0,1
9997,1221523.5,0.0,9,30.1,823305.0,0,14,0,13.675946,12.083377,...,0,0,0,0,0,0,0,0,0,1
9998,3343584.0,0.0,10,20.3,724314.0,0,25,0,13.001346,11.844353,...,0,0,0,0,0,0,0,0,1,0


In [17]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, Y, random_state=0)
eval_set = [(X_train, y_train), (X_val, y_val)]
# eval_set: 검증 데이터, verbose: 학습 중 평가 결과 출력 여부

In [18]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(
    n_esimators=400,
    learning_rate=0.7,
    early_stopping_rounds=100,
    verbose=1
)
eval_set = [(X_tr, y_tr), (X_val, y_val)]
lgbm.fit(X_tr, y_tr, eval_set=eval_set)


[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Info] Number of positive: 2595, number of negative: 4905
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001738 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1876
[LightGBM] [Info] Number of data points in the train set: 7500, number of used features: 38
[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.346

LGBMClassifier(early_stopping_rounds=100, learning_rate=0.7, n_esimators=400,
               verbose=1)

In [19]:
# lgbm.score(X_train, y_train), lgbm.score(X_test, y_test)

In [20]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Optuna 최적화 함수 정의
def optimize_lightgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
    }

    # LightGBM 모델 생성
    model = lgb.LGBMClassifier(**params, random_state=42)
    model.fit(X_train, y_train)

    # 예측 및 성능 평가
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)  # 정확도를 최적화 목표로 설정

# Optuna 스터디 생성 및 최적화 실행
study_lightgbm = optuna.create_study(direction="maximize")
study_lightgbm.optimize(optimize_lightgbm, n_trials=50)  # 50번 탐색

# 최적의 하이퍼파라미터 출력
best_params_lightgbm = study_lightgbm.best_params
print("Best LightGBM Hyperparameters:", best_params_lightgbm)

# 최적의 하이퍼파라미터를 적용한 모델 학습
best_lightgbm = lgb.LGBMClassifier(**best_params_lightgbm, random_state=42)
best_lightgbm.fit(X_train, y_train)

# 최적 모델 평가
y_pred_best = best_lightgbm.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred_best)
print("Final Optimized LightGBM Accuracy:", final_accuracy)


[I 2025-03-20 16:38:34,600] A new study created in memory with name: no-name-61553159-c8d5-445a-b41f-c1382cbc7632
[W 2025-03-20 16:38:34,770] Trial 0 failed with parameters: {'n_estimators': 241, 'learning_rate': 0.039517921237995676, 'max_depth': 7, 'num_leaves': 24, 'min_child_samples': 23, 'subsample': 0.7313408910812658, 'colsample_bytree': 0.9417257925019127, 'reg_alpha': 4.328234448614113, 'reg_lambda': 8.953770994474777, 'min_split_gain': 0.7168234694466722} because of the following error: NameError("name 'X_test' is not defined").
Traceback (most recent call last):
  File "/Users/ozzt/opt/anaconda3/envs/dacon/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/38/cm7w0qmn1l73yjcy3j0z1x6w0000gn/T/ipykernel_35112/3384970397.py", line 27, in optimize_lightgbm
    y_pred = model.predict(X_test)
NameError: name 'X_test' is not defined
[W 2025-03-20 16:38:34,775] Trial 0 failed with value None.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2595, number of negative: 4905
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1874
[LightGBM] [Info] Number of data points in the train set: 7500, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.346000 -> initscore=-0.636669
[LightGBM] [Info] Start training from score -0.636669
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

NameError: name 'X_test' is not defined

In [14]:
from joblib import dump

dump(best_lightgbm, './best_model/light_GBM.joblib')

['./best_model/light_GBM.joblib']

In [15]:
best_lightgbm.score(X_train, y_train), best_lightgbm.score(X_test, y_test)

(0.7724, 0.7248)

In [17]:
# 예측
y_pred_train = best_lightgbm.predict(X_train)
y_pred_test = best_lightgbm.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score, classification_report

print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_test, y_pred_test))

print(classification_report(y_test, y_pred_test))

0.7724
0.7248
              precision    recall  f1-score   support

           0       0.74      0.89      0.81      1618
           1       0.68      0.42      0.52       882

    accuracy                           0.72      2500
   macro avg       0.71      0.66      0.66      2500
weighted avg       0.72      0.72      0.71      2500



In [ ]:
# 예측
y_pred = best_lightgbm.predict(X_test)
y_proba = best_lightgbm.predict_proba(X_test)[:, 1]  # ROC Curve 및 PR Curve를 위한 확률값

# 성능 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

# 결과 출력
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1-Score: {f1:.4f}")
print(f"✅ AUC-ROC: {roc_auc:.4f}")